In [10]:
import tweepy
import pandas as pd
import time
import sys

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABG55gEAAAAAbpWAofmkLpaS%2Bz1AAIyzhncEdJU%3D9bBu8UjFMR64nLkBiwaTXt1XqquiCrFGGNsdgnZRj7VC3Z3kQA"

try:
    client = tweepy.Client(BEARER_TOKEN)
except Exception as e:
    print(f"Gagal menginisialisasi klien Tweepy. Cek Bearer Token Anda: {e}")
    sys.exit(1)

keywords = [
    "bangsat", "kontol", "memek", "babi", "goblok",
    "tolol", "ngentot", "brengsek", "tai", "jijik"
]
limit_per_keyword = 10  # Batasan sesuai permintaan Anda
all_tweets = []

print("Memulai Scraping menggunakan Twitter API v2...")

for kw in keywords:
    query = f'"{kw}" lang:id -is:retweet'
    print(f"\nScraping: {query} (Target: {limit_per_keyword} tweets)")

    try:
        response = client.search_recent_tweets(
            query=query,
            max_results=limit_per_keyword,
            # Menambahkan field untuk mengambil data tambahan
            tweet_fields=["created_at", "text"],
            expansions=["author_id"],
            user_fields=["username"]
        )

        # Memproses data untuk digabungkan
        # Membuat kamus (dictionary) dari ID pengguna ke objek pengguna untuk akses cepat
        users = {user.id: user for user in response.includes['users']} if response.includes and 'users' in response.includes else {}
        
        if response.data:
            for tweet in response.data:
                # Mengambil username
                user_obj = users.get(tweet.author_id)
                username = user_obj.username if user_obj else 'N/A'
                
                all_tweets.append([
                    tweet.created_at, 
                    username, 
                    tweet.text, 
                    kw
                ])
            print(f"  -> Berhasil mengumpulkan {len(response.data)} tweet.")
        else:
            print("  -> Tidak ada tweet yang ditemukan untuk kata kunci ini dalam 7 hari terakhir.")

    except tweepy.errors.TooManyRequests as e:
        print(f"  -> ERROR Rate Limit: Terlalu banyak permintaan. Jeda 15 menit. {e}")
        break # Berhenti jika rate limit tercapai
    except Exception as e:
        print(f"  -> ERROR saat scrape {kw}: {e}")
        time.sleep(2) # Jeda untuk menghindari rate limit yang terlalu cepat
        continue
    
    time.sleep(3)

# ==============================================================================
# 5. MEMBUAT DAN MENYIMPAN DATAFRAME
# ==============================================================================
df = pd.DataFrame(all_tweets, columns=["date", "user", "tweet", "keyword"])
df = df.drop_duplicates(subset=["tweet"]) 

filename = "hate_speech_tweepy_50.csv"
df.to_csv(filename, index=False)

print("\n--- Selesai ---")
print("Total collected (unique tweets):", len(df))
print(f"Data disimpan ke '{filename}'")

Memulai Scraping menggunakan Twitter API v2...

Scraping: "bangsat" lang:id -is:retweet (Target: 10 tweets)
  -> Berhasil mengumpulkan 10 tweet.

Scraping: "kontol" lang:id -is:retweet (Target: 10 tweets)
  -> ERROR Rate Limit: Terlalu banyak permintaan. Jeda 15 menit. 429 Too Many Requests
Too Many Requests

--- Selesai ---
Total collected (unique tweets): 10
Data disimpan ke 'hate_speech_tweepy_50.csv'
